In [ ]:
# %matplotlib inline
# import matplotlib.pyplot as plt
# import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow.keras import backend as k
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding, Dropout, LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import model_from_json

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

import pickle

# print(tf.__version__)
# print(tf.keras.__version__)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# This code is partly based on a tutorial which can be found here:
# https://fasttext.cc/docs/en/supervised-tutorial.html


In [ ]:
#Packages to import (regular expressions and pandas)

import re 
import pandas as pd 

from pathlib import Path
import gzip
import json

# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#Mount google colab to google drive so it can access files
from google.colab import drive
drive.mount('/content/drive')

#The huge gzipped JSON files ('NL_jobs.15.jsonl.gz') are located in this folder
cwd = '/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments'

#Print the content of the folder
!ls '/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments'

# set current working directory, This folder should only contain the huge json files.
%cd '/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments'


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
'Baseline Model (BOW Log. regression).ipynb'
 BERT_alt_deploy.ipynb
'BERT_Jads_Nikita- Clyde edit - hpartuning.ipynb'
 BOW_LSTM.ipynb
'Copy of Copy of BERT_alt _ hyperparamtuning.ipynb'
 CORPUSNL_jobs.15.jsonl.gz_JSON_splitted.tsv_1.tsv
 Cow
 COW_LSTM_hyperparamtuning.ipynb
 dev.gsheet
 dev.tsv
 df3_updated_clean.txt
 df3_updated.txt
 df4_updated_clean.txt
 df4_updated.txt
 Exp23_RNDSCV.csv
 Exp25_RNDSCV.csv
 Exp26_RNDSCV.csv
 Exp27_RNDSCV.csv
 Exp28_RNDSCV.cs

In [ ]:
TRAIN_FILE = "train.tsv"
DEV_FILE = "dev.tsv"

print("TRAIN FILE: \t\t{}\nDEVELOPMENT FILE: \t{}".format(TRAIN_FILE, DEV_FILE))

TRAIN FILE: 		train.tsv
DEVELOPMENT FILE: 	dev.tsv


In [ ]:
# Load the data
test_full = pd.read_csv(TRAIN_FILE, sep = "\t")
dev_full = pd.read_csv(DEV_FILE, sep = "\t")

# Tirm the data to only one we need
test_trim = test_full[["Quality", "#1 String"]]
dev_trim = dev_full[["Quality", "#1 String"]]

# Seperate our data for training and testing
train_text = np.array(test_full["#1 String"])
train_target = np.array(test_full["Quality"])

test_text = np.array(dev_full["#1 String"])
test_target = np.array(dev_full["Quality"])

# Check our training and testing sets
print("Train-set size: ", len(train_text))
print("Test-set size:  ", len(test_text))

# Stack text for tokenization
data_text = np.hstack((train_text, test_text))
print("Shape of the stacked text: {}".format(data_text.shape))

Train-set size:  2240
Test-set size:   720
Shape of the stacked text: (2960,)


!unzip v0.1.0.zip

In [ ]:
new_items = ['noskill' if x == 0  else 'skill' for x in train_target]
print(new_items[0:10])

df3 = pd.DataFrame( {'text':train_text, 'target':new_items})


new_items2 = ['noskill' if x == 0  else 'skill' for x in test_target]
print(new_items2[0:10])

df4 = pd.DataFrame( {'text':test_text, 'target':new_items2})


['noskill', 'noskill', 'noskill', 'noskill', 'skill', 'skill', 'noskill', 'noskill', 'noskill', 'skill']
['skill', 'skill', 'skill', 'skill', 'noskill', 'noskill', 'noskill', 'noskill', 'noskill', 'noskill']


In [ ]:
import pandas as pd
from io import StringIO
import csv

col = ['target', 'text']

df3 = df3[col]
df3['target']=['__label__'+ s for s in df3['target']]
df3['text']= df3['text'].replace('\n',' ', regex=True).replace('\t',' ', regex=True)
df3.to_csv(r'/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments/df3_updated.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")


df4 = df4[col]
df4['target']=['__label__'+ s for s in df4['target']]
df4['text']= df4['text'].replace('\n',' ', regex=True).replace('\t',' ', regex=True)
df4.to_csv(r'/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments/df4_updated.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

In [ ]:
print(df3.head())
print(df4.head())

             target                                               text
0  __label__noskill                                                  0
1  __label__noskill                                Functieomschrijving
2  __label__noskill               Schoonmaken van kantoren en scholen.
3  __label__noskill  Bestellen schoonmaakproducten en benodigdheden...
4    __label__skill              Ruime ervaring als interieurverzorger
             target                                               text
0    __label__skill  Je screent naar behoeften en vertaalt deze doo...
1    __label__skill  Waar mogelijk blijf je nieuwe projecten zelf l...
2    __label__skill  Je bent creatief en koppelt graag opgaven, zod...
3    __label__skill  Je bent in staat om multidisciplinaire project...
4  __label__noskill  Iris Personeelsdiensten is een Werving & Selec...


In [ ]:
!pip install fasttext

     |████████████████████████████████| 71kB 2.5MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3015946 sha256=dfc8e7ebc78b5d9a626d068bdfd02e054c4aa73cef61cc4e450cca25c73366e5
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [ ]:
import fasttext

model1 = fasttext.train_supervised('/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments/df3_updated.txt')

def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

print_results(*model1.test('/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments/df4_updated.txt'))



import pandas as pd
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as sk

predictionLabelList = []
predictionProbList = []
for x in range(len(df4['text'])):
  prediction = model1.predict(df4['text'][x], k=2)
  predictionLabelList.append(prediction[0])

  predictionProbList.append(prediction[1])

# print(predictionLabelList)
# print(predictionProbList)

# df = pd.DataFrame([predictionLabelList[x][0],  predictionProbList[x][0]] for x in range(len(predictionLabelList))  )

df = pd.DataFrame([predictionLabelList[x][0]] for x in range(len(predictionLabelList))  )
df = df[0].replace('__label__noskill',0, regex=True).replace('__label__skill',1, regex=True)
# df = pd.DataFrame(
#     [df4['target'][x], df4['text'][x], model1.predict(df4['text'][x], k=2)] for x in range(len(df4['text']))
# )

eval_predicted = df.values

# test acc

no_predictions_eval = len(eval_predicted)
# print(no_predictions_eval)

right_eval = np.sum(eval_predicted == test_target)

test_accuracy = right_eval/ no_predictions_eval
print("Test Accuracy: ", test_accuracy)


precision = sk.precision_score(test_target, eval_predicted)
print("Eval Precision: ", precision)
recall = sk.recall_score(test_target, eval_predicted)
print("Eval recall: ", recall)
roc_auc = sk.roc_auc_score(test_target, eval_predicted)
print("Eval roc_auc: ", roc_auc)
f1 = sk.f1_score(test_target, eval_predicted)
print("Eval f1: ", f1)

N	720
P@1	0.754
R@1	0.754
Test Accuracy:  0.7541666666666667
Eval Precision:  0.6513157894736842
Eval recall:  0.7360594795539034
Eval roc_auc:  0.7505131100651999
Eval f1:  0.6910994764397905


In [ ]:
!cat "/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments/df3_updated.txt" | sed -e "s/\([.\!?,’/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > "/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments/df3_updated_clean.txt"
!cat "/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments/df4_updated.txt" | sed -e "s/\([.\!?,’/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > "/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments/df4_updated_clean.txt"

In [ ]:
import fasttext

model1 = fasttext.train_supervised('/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments/df3_updated_clean.txt')

def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

print_results(*model1.test('/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments/df4_updated_clean.txt'))



import pandas as pd
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as sk

predictionLabelList = []
predictionProbList = []
for x in range(len(df4['text'])):
  prediction = model1.predict(df4['text'][x], k=2)
  predictionLabelList.append(prediction[0])

  predictionProbList.append(prediction[1])

# print(predictionLabelList)
# print(predictionProbList)

# df = pd.DataFrame([predictionLabelList[x][0],  predictionProbList[x][0]] for x in range(len(predictionLabelList))  )

df = pd.DataFrame([predictionLabelList[x][0]] for x in range(len(predictionLabelList))  )
df = df[0].replace('__label__noskill',0, regex=True).replace('__label__skill',1, regex=True)
# df = pd.DataFrame(
#     [df4['target'][x], df4['text'][x], model1.predict(df4['text'][x], k=2)] for x in range(len(df4['text']))
# )

eval_predicted = df.values

# test acc

no_predictions_eval = len(eval_predicted)
# print(no_predictions_eval)

right_eval = np.sum(eval_predicted == test_target)

test_accuracy = right_eval/ no_predictions_eval
print("Test Accuracy: ", test_accuracy)


precision = sk.precision_score(test_target, eval_predicted)
print("Eval Precision: ", precision)
recall = sk.recall_score(test_target, eval_predicted)
print("Eval recall: ", recall)
roc_auc = sk.roc_auc_score(test_target, eval_predicted)
print("Eval roc_auc: ", roc_auc)
f1 = sk.f1_score(test_target, eval_predicted)
print("Eval f1: ", f1)

N	720
P@1	0.768
R@1	0.768
Test Accuracy:  0.7069444444444445
Eval Precision:  0.5917721518987342
Eval recall:  0.6951672862453532
Eval roc_auc:  0.7045681220583752
Eval f1:  0.6393162393162393


In [ ]:
import fasttext

model1 = fasttext.train_supervised('/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments/df3_updated_clean.txt', lr=0.5, epoch=30)



def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

print_results(*model1.test('/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments/df4_updated_clean.txt'))



import pandas as pd
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as sk

predictionLabelList = []
predictionProbList = []
for x in range(len(df4['text'])):
  prediction = model1.predict(df4['text'][x], k=2)
  predictionLabelList.append(prediction[0])

  predictionProbList.append(prediction[1])

# print(predictionLabelList)
# print(predictionProbList)

# df = pd.DataFrame([predictionLabelList[x][0],  predictionProbList[x][0]] for x in range(len(predictionLabelList))  )

df = pd.DataFrame([predictionLabelList[x][0]] for x in range(len(predictionLabelList))  )
df = df[0].replace('__label__noskill',0, regex=True).replace('__label__skill',1, regex=True)
# df = pd.DataFrame(
#     [df4['target'][x], df4['text'][x], model1.predict(df4['text'][x], k=2)] for x in range(len(df4['text']))
# )

eval_predicted = df.values

# test acc

no_predictions_eval = len(eval_predicted)
# print(no_predictions_eval)

right_eval = np.sum(eval_predicted == test_target)

test_accuracy = right_eval/ no_predictions_eval
print("Test Accuracy: ", test_accuracy)


precision = sk.precision_score(test_target, eval_predicted)
print("Eval Precision: ", precision)
recall = sk.recall_score(test_target, eval_predicted)
print("Eval recall: ", recall)
roc_auc = sk.roc_auc_score(test_target, eval_predicted)
print("Eval roc_auc: ", roc_auc)
f1 = sk.f1_score(test_target, eval_predicted)
print("Eval f1: ", f1)

N	720
P@1	0.772
R@1	0.772
Test Accuracy:  0.7458333333333333
Eval Precision:  0.6335403726708074
Eval recall:  0.758364312267658
Eval roc_auc:  0.7483617570207469
Eval f1:  0.6903553299492385


In [ ]:
import fasttext

model1 = fasttext.train_supervised('/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments/df3_updated_clean.txt', lr=0.5, epoch=100)



def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

print_results(*model1.test('/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments/df4_updated_clean.txt'))



import pandas as pd
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as sk

predictionLabelList = []
predictionProbList = []
for x in range(len(df4['text'])):
  prediction = model1.predict(df4['text'][x], k=2)
  predictionLabelList.append(prediction[0])

  predictionProbList.append(prediction[1])

# print(predictionLabelList)
# print(predictionProbList)

# df = pd.DataFrame([predictionLabelList[x][0],  predictionProbList[x][0]] for x in range(len(predictionLabelList))  )

df = pd.DataFrame([predictionLabelList[x][0]] for x in range(len(predictionLabelList))  )
df = df[0].replace('__label__noskill',0, regex=True).replace('__label__skill',1, regex=True)
# df = pd.DataFrame(
#     [df4['target'][x], df4['text'][x], model1.predict(df4['text'][x], k=2)] for x in range(len(df4['text']))
# )

eval_predicted = df.values

# test acc

no_predictions_eval = len(eval_predicted)
# print(no_predictions_eval)

right_eval = np.sum(eval_predicted == test_target)

test_accuracy = right_eval/ no_predictions_eval
print("Test Accuracy: ", test_accuracy)


precision = sk.precision_score(test_target, eval_predicted)
print("Eval Precision: ", precision)
recall = sk.recall_score(test_target, eval_predicted)
print("Eval recall: ", recall)
roc_auc = sk.roc_auc_score(test_target, eval_predicted)
print("Eval roc_auc: ", roc_auc)
f1 = sk.f1_score(test_target, eval_predicted)
print("Eval f1: ", f1)

N	720
P@1	0.782
R@1	0.782
Test Accuracy:  0.7486111111111111
Eval Precision:  0.6349693251533742
Eval recall:  0.7695167286245354
Eval roc_auc:  0.7528293177490747
Eval f1:  0.6957983193277311


In [ ]:
import fasttext

model1 = fasttext.train_supervised('/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments/df3_updated_clean.txt', lr=0.5, epoch=30, wordNgrams=2)



def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

print_results(*model1.test('/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments/df4_updated_clean.txt'))



import pandas as pd
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as sk

predictionLabelList = []
predictionProbList = []
for x in range(len(df4['text'])):
  prediction = model1.predict(df4['text'][x], k=2)
  predictionLabelList.append(prediction[0])

  predictionProbList.append(prediction[1])

# print(predictionLabelList)
# print(predictionProbList)

# df = pd.DataFrame([predictionLabelList[x][0],  predictionProbList[x][0]] for x in range(len(predictionLabelList))  )

df = pd.DataFrame([predictionLabelList[x][0]] for x in range(len(predictionLabelList))  )
df = df[0].replace('__label__noskill',0, regex=True).replace('__label__skill',1, regex=True)
# df = pd.DataFrame(
#     [df4['target'][x], df4['text'][x], model1.predict(df4['text'][x], k=2)] for x in range(len(df4['text']))
# )

eval_predicted = df.values

# test acc

no_predictions_eval = len(eval_predicted)
# print(no_predictions_eval)

right_eval = np.sum(eval_predicted == test_target)

test_accuracy = right_eval/ no_predictions_eval
print("Test Accuracy: ", test_accuracy)


precision = sk.precision_score(test_target, eval_predicted)
print("Eval Precision: ", precision)
recall = sk.recall_score(test_target, eval_predicted)
print("Eval recall: ", recall)
roc_auc = sk.roc_auc_score(test_target, eval_predicted)
print("Eval roc_auc: ", roc_auc)
f1 = sk.f1_score(test_target, eval_predicted)
print("Eval f1: ", f1)

N	720
P@1	0.769
R@1	0.769
Test Accuracy:  0.7416666666666667
Eval Precision:  0.6309148264984227
Eval recall:  0.7434944237918215
Eval roc_auc:  0.7420354602329395
Eval f1:  0.6825938566552902


In [ ]:
import fasttext

model1 = fasttext.train_supervised('/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments/df3_updated_clean.txt', lr=1, epoch=30, wordNgrams=2)



def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

print_results(*model1.test('/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments/df4_updated_clean.txt'))



import pandas as pd
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as sk

predictionLabelList = []
predictionProbList = []
for x in range(len(df4['text'])):
  prediction = model1.predict(df4['text'][x], k=2)
  predictionLabelList.append(prediction[0])

  predictionProbList.append(prediction[1])

# print(predictionLabelList)
# print(predictionProbList)

# df = pd.DataFrame([predictionLabelList[x][0],  predictionProbList[x][0]] for x in range(len(predictionLabelList))  )

df = pd.DataFrame([predictionLabelList[x][0]] for x in range(len(predictionLabelList))  )
df = df[0].replace('__label__noskill',0, regex=True).replace('__label__skill',1, regex=True)
# df = pd.DataFrame(
#     [df4['target'][x], df4['text'][x], model1.predict(df4['text'][x], k=2)] for x in range(len(df4['text']))
# )

eval_predicted = df.values

# test acc

no_predictions_eval = len(eval_predicted)
# print(no_predictions_eval)

right_eval = np.sum(eval_predicted == test_target)

test_accuracy = right_eval/ no_predictions_eval
print("Test Accuracy: ", test_accuracy)


precision = sk.precision_score(test_target, eval_predicted)
print("Eval Precision: ", precision)
recall = sk.recall_score(test_target, eval_predicted)
print("Eval recall: ", recall)
roc_auc = sk.roc_auc_score(test_target, eval_predicted)
print("Eval roc_auc: ", roc_auc)
f1 = sk.f1_score(test_target, eval_predicted)
print("Eval f1: ", f1)

N	720
P@1	0.776
R@1	0.776
Test Accuracy:  0.7388888888888889
Eval Precision:  0.6293929712460063
Eval recall:  0.7323420074349443
Eval roc_auc:  0.7375678995046119
Eval f1:  0.6769759450171821


In [ ]:
import fasttext

model1 = fasttext.train_supervised('/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments/df3_updated_clean.txt', lr=0.8, epoch=30, wordNgrams=2)



def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

print_results(*model1.test('/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments/df4_updated_clean.txt'))



import pandas as pd
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as sk

predictionLabelList = []
predictionProbList = []
for x in range(len(df4['text'])):
  prediction = model1.predict(df4['text'][x], k=2)
  predictionLabelList.append(prediction[0])

  predictionProbList.append(prediction[1])

# print(predictionLabelList)
# print(predictionProbList)

# df = pd.DataFrame([predictionLabelList[x][0],  predictionProbList[x][0]] for x in range(len(predictionLabelList))  )

df = pd.DataFrame([predictionLabelList[x][0]] for x in range(len(predictionLabelList))  )
df = df[0].replace('__label__noskill',0, regex=True).replace('__label__skill',1, regex=True)
# df = pd.DataFrame(
#     [df4['target'][x], df4['text'][x], model1.predict(df4['text'][x], k=2)] for x in range(len(df4['text']))
# )

eval_predicted = df.values

# test acc

no_predictions_eval = len(eval_predicted)
# print(no_predictions_eval)

right_eval = np.sum(eval_predicted == test_target)

test_accuracy = right_eval/ no_predictions_eval
print("Test Accuracy: ", test_accuracy)


precision = sk.precision_score(test_target, eval_predicted)
print("Eval Precision: ", precision)
recall = sk.recall_score(test_target, eval_predicted)
print("Eval recall: ", recall)
roc_auc = sk.roc_auc_score(test_target, eval_predicted)
print("Eval roc_auc: ", roc_auc)
f1 = sk.f1_score(test_target, eval_predicted)
print("Eval f1: ", f1)

N	720
P@1	0.768
R@1	0.768
Test Accuracy:  0.7402777777777778
Eval Precision:  0.629746835443038
Eval recall:  0.7397769516728625
Eval roc_auc:  0.74017672417346
Eval f1:  0.6803418803418804


In [ ]:
import fasttext

model1 = fasttext.train_supervised('/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments/df3_updated.txt', wordNgrams=2)



def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

print_results(*model1.test('/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments/df4_updated.txt'))



import pandas as pd
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as sk

predictionLabelList = []
predictionProbList = []
for x in range(len(df4['text'])):
  prediction = model1.predict(df4['text'][x], k=2)
  predictionLabelList.append(prediction[0])

  predictionProbList.append(prediction[1])

# print(predictionLabelList)
# print(predictionProbList)

# df = pd.DataFrame([predictionLabelList[x][0],  predictionProbList[x][0]] for x in range(len(predictionLabelList))  )

df = pd.DataFrame([predictionLabelList[x][0]] for x in range(len(predictionLabelList))  )
df = df[0].replace('__label__noskill',0, regex=True).replace('__label__skill',1, regex=True)
# df = pd.DataFrame(
#     [df4['target'][x], df4['text'][x], model1.predict(df4['text'][x], k=2)] for x in range(len(df4['text']))
# )

eval_predicted = df.values

# test acc

no_predictions_eval = len(eval_predicted)
# print(no_predictions_eval)

right_eval = np.sum(eval_predicted == test_target)

test_accuracy = right_eval/ no_predictions_eval
print("Test Accuracy: ", test_accuracy)


precision = sk.precision_score(test_target, eval_predicted)
print("Eval Precision: ", precision)
recall = sk.recall_score(test_target, eval_predicted)
print("Eval recall: ", recall)
roc_auc = sk.roc_auc_score(test_target, eval_predicted)
print("Eval roc_auc: ", roc_auc)
f1 = sk.f1_score(test_target, eval_predicted)
print("Eval f1: ", f1)

N	720
P@1	0.711
R@1	0.711
Test Accuracy:  0.7111111111111111
Eval Precision:  0.5868945868945868
Eval recall:  0.7657992565055762
Eval roc_auc:  0.7221457479867127
Eval f1:  0.6645161290322581


In [ ]:
import fasttext

model1 = fasttext.train_supervised('/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments/df3_updated_clean.txt', lr=0.5, epoch=100, wordNgrams=2)



def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

print_results(*model1.test('/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments/df4_updated_clean.txt'))



import pandas as pd
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as sk

predictionLabelList = []
predictionProbList = []
for x in range(len(df4['text'])):
  prediction = model1.predict(df4['text'][x], k=2)
  predictionLabelList.append(prediction[0])

  predictionProbList.append(prediction[1])

# print(predictionLabelList)
# print(predictionProbList)

# df = pd.DataFrame([predictionLabelList[x][0],  predictionProbList[x][0]] for x in range(len(predictionLabelList))  )

df = pd.DataFrame([predictionLabelList[x][0]] for x in range(len(predictionLabelList))  )
df = df[0].replace('__label__noskill',0, regex=True).replace('__label__skill',1, regex=True)
# df = pd.DataFrame(
#     [df4['target'][x], df4['text'][x], model1.predict(df4['text'][x], k=2)] for x in range(len(df4['text']))
# )

eval_predicted = df.values

# test acc

no_predictions_eval = len(eval_predicted)
# print(no_predictions_eval)

right_eval = np.sum(eval_predicted == test_target)

test_accuracy = right_eval/ no_predictions_eval
print("Test Accuracy: ", test_accuracy)


precision = sk.precision_score(test_target, eval_predicted)
print("Eval Precision: ", precision)
recall = sk.recall_score(test_target, eval_predicted)
print("Eval recall: ", recall)
roc_auc = sk.roc_auc_score(test_target, eval_predicted)
print("Eval roc_auc: ", roc_auc)
f1 = sk.f1_score(test_target, eval_predicted)
print("Eval f1: ", f1)

N	720
P@1	0.772
R@1	0.772
Test Accuracy:  0.7347222222222223
Eval Precision:  0.6274509803921569
Eval recall:  0.7137546468401487
Eval roc_auc:  0.7304915141074358
Eval f1:  0.6678260869565217


In [ ]:
import fasttext

model1 = fasttext.train_supervised('/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments/df3_updated_clean.txt', lr=0.5, epoch=100, wordNgrams=4)



def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

print_results(*model1.test('/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments/df4_updated_clean.txt'))



import pandas as pd
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as sk

predictionLabelList = []
predictionProbList = []
for x in range(len(df4['text'])):
  prediction = model1.predict(df4['text'][x], k=2)
  predictionLabelList.append(prediction[0])

  predictionProbList.append(prediction[1])

# print(predictionLabelList)
# print(predictionProbList)

# df = pd.DataFrame([predictionLabelList[x][0],  predictionProbList[x][0]] for x in range(len(predictionLabelList))  )

df = pd.DataFrame([predictionLabelList[x][0]] for x in range(len(predictionLabelList))  )
df = df[0].replace('__label__noskill',0, regex=True).replace('__label__skill',1, regex=True)
# df = pd.DataFrame(
#     [df4['target'][x], df4['text'][x], model1.predict(df4['text'][x], k=2)] for x in range(len(df4['text']))
# )

eval_predicted = df.values

# test acc

no_predictions_eval = len(eval_predicted)
# print(no_predictions_eval)

right_eval = np.sum(eval_predicted == test_target)

test_accuracy = right_eval/ no_predictions_eval
print("Test Accuracy: ", test_accuracy)


precision = sk.precision_score(test_target, eval_predicted)
print("Eval Precision: ", precision)
recall = sk.recall_score(test_target, eval_predicted)
print("Eval recall: ", recall)
roc_auc = sk.roc_auc_score(test_target, eval_predicted)
print("Eval roc_auc: ", roc_auc)
f1 = sk.f1_score(test_target, eval_predicted)
print("Eval f1: ", f1)

N	720
P@1	0.771
R@1	0.771
Test Accuracy:  0.7236111111111111
Eval Precision:  0.6114649681528662
Eval recall:  0.7137546468401487
Eval roc_auc:  0.7216223345065488
Eval f1:  0.6586620926243567


In [ ]:
model1.predict(df4['text'][0], k=2)

In [ ]:
print(df4.head())

In [ ]:
# Auto tune
!./fasttext supervised -input train.txt -output model -autotune-validation valid.txt -autotune-duration 600